# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [2]:
# data has been downloaded from UC Irvine machine learning website
spark = SparkSession.builder.appName('lr_hyundai').getOrCreate()
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [3]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [11]:
from pyspark.ml.feature import StringIndexer
str_indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_indexed')
indexed = str_indexer.fit(data).transform(data)
assembler = VectorAssembler(inputCols=['Tonnage', 'passengers', 'length', 'cabins', 
                                       'passenger_density', 'Cruise_line_indexed', 'Age'],
                           outputCol='features')
output = assembler.transform(indexed)
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_indexed: double (nullable = false)
 |-- features: vector (nullable = true)



In [13]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[30.2769999999999...|
|[30.2769999999999...|
|[47.262,14.86,7.2...|
|[110.0,29.74,9.53...|
|[101.353,26.42,8....|
|[70.367,20.52,8.5...|
|[70.367,20.52,8.5...|
|[70.367,20.56,8.5...|
|[70.367,20.52,8.5...|
|[110.238999999999...|
|[110.0,29.74,9.51...|
|[46.052,14.52,7.2...|
|[70.367,20.52,8.5...|
|[70.367,20.52,8.5...|
|[86.0,21.24,9.63,...|
|[110.0,29.74,9.51...|
|[88.5,21.24,9.63,...|
|[70.367,20.52,8.5...|
|[88.5,21.24,9.63,...|
|[70.367,20.52,8.5...|
+--------------------+
only showing top 20 rows



In [22]:
final_data = output.select(['features', 'crew'])
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[30.2769999999999...|3.55|
|[30.2769999999999...|3.55|
|[47.262,14.86,7.2...| 6.7|
|[110.0,29.74,9.53...|19.1|
|[101.353,26.42,8....|10.0|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.56,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[110.238999999999...|11.5|
|[110.0,29.74,9.51...|11.6|
|[46.052,14.52,7.2...| 6.6|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[86.0,21.24,9.63,...| 9.3|
|[110.0,29.74,9.51...|11.6|
|[88.5,21.24,9.63,...|10.3|
|[70.367,20.52,8.5...| 9.2|
|[88.5,21.24,9.63,...| 9.3|
|[70.367,20.52,8.5...| 9.2|
+--------------------+----+
only showing top 20 rows



### Splitting up training and testing data

In [30]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               107|
|   mean| 7.705700934579442|
| stddev|3.4712591136710618|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



### Creating the linear regression model and training and testing the data

In [31]:
lr_ship = LinearRegression(featuresCol='features', labelCol='crew', predictionCol='predicted crew')
trained_ship_model = lr_ship.fit(train_data)

In [32]:
ship_test_results = trained_ship_model.evaluate(test_data)

In [33]:
ship_test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.27371424410699696|
| 0.28356577477267686|
| -1.0554455037334227|
|-0.12152641845214696|
|0.028889439248068527|
|-0.00421998765122...|
|  0.3854095636321513|
| -0.7715432441404388|
| 0.12000462740902584|
|-0.05851487698840...|
|  0.6038302149410901|
|  0.9001025011660939|
|-0.05895500911088991|
|  0.4703259911194415|
| -0.5984370521787676|
|-0.19910625718605512|
| -0.6964025991518321|
| 0.06431735364602353|
| 0.06963064903835736|
| -1.3606530821648333|
+--------------------+
only showing top 20 rows



### Evaluating the model

In [34]:
ship_test_results.rootMeanSquaredError

0.6215453209719183

In [35]:
ship_test_results.r2

0.9695589785486132

In [39]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               107|
|   mean| 7.705700934579442|
| stddev|3.4712591136710618|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



#### However, when you get a good root mean squared error and r-sqaured value, we should always do a reality check on the model. Does it make sense that you're getting this good of a model? 
#### To do a reality check on this, we could check on these results by checking if any 2 columns are highlt correlated. For example, is the # of passengers on board highly correlated to # of crew? is the # of crew highy correlated to # of cabins?

#### If they are highly correlated, it makes sense that the model performed really well because there are 2 columns that highly correlated.

In [42]:
from pyspark.sql.functions import corr
data.select(corr('passengers', 'crew').alias('corr(crew, passengers)')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



#### The result above makes sense intuitively because more passengers would need more crew.